In [1]:
import findspark
findspark.init()
findspark.find()

'C:\\Users\\HAL 9004\\.conda\\envs\\spark-polito\\lib\\site-packages\\pyspark'

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("Excercise 33") \
    .getOrCreate()

In [3]:
spark

In [4]:
#Exercise 32

# Maximum value
# Input: a collection of (structured) textual csv files containing the daily value of PM10 for a set of sensors
#   Each line of the files has the following format: sensorId,date,PM10 value (μg/m3)\n
# Output: report the maximum value of PM10
#   Print the result on the standard output

## With RDD's

In [4]:
sc = spark.sparkContext

In [5]:
sc

<SparkContext master=local[*] appName=Excercise 33>

In [29]:
rdd_33 = sc.textFile("input_file_33.txt")

In [30]:
rdd_33.take(30)

['s1,2016-01-01,20.5',
 's2,2016-01-01,30.1',
 's1,2016-01-02,60.2',
 's2,2016-01-02,20.4',
 's1,2016-01-03,55.5',
 's2,2016-01-03,52.5']

In [31]:
rdd_33 = rdd_33.map(lambda a: a.split(","))

In [32]:
rdd_33.take(30)

[['s1', '2016-01-01', '20.5'],
 ['s2', '2016-01-01', '30.1'],
 ['s1', '2016-01-02', '60.2'],
 ['s2', '2016-01-02', '20.4'],
 ['s1', '2016-01-03', '55.5'],
 ['s2', '2016-01-03', '52.5']]

In [33]:
rdd_33 = rdd_33.map(lambda a: (a[2],(a[0],a[1])) ) 
rdd_33.take(10)

[('20.5', ('s1', '2016-01-01')),
 ('30.1', ('s2', '2016-01-01')),
 ('60.2', ('s1', '2016-01-02')),
 ('20.4', ('s2', '2016-01-02')),
 ('55.5', ('s1', '2016-01-03')),
 ('52.5', ('s2', '2016-01-03'))]

In [37]:
rdd_33 = rdd_33.sortByKey(ascending = False)
rdd_33.take(10)

[('60.2', ('s1', '2016-01-02')),
 ('55.5', ('s1', '2016-01-03')),
 ('52.5', ('s2', '2016-01-03')),
 ('30.1', ('s2', '2016-01-01')),
 ('20.5', ('s1', '2016-01-01')),
 ('20.4', ('s2', '2016-01-02'))]

## With DataFrames

In [39]:
df33 = spark.read.csv("input_file_33.txt")

In [40]:
df33

DataFrame[_c0: string, _c1: string, _c2: string]

In [41]:
df33.take(10)

[Row(_c0='s1', _c1='2016-01-01', _c2='20.5'),
 Row(_c0='s2', _c1='2016-01-01', _c2='30.1'),
 Row(_c0='s1', _c1='2016-01-02', _c2='60.2'),
 Row(_c0='s2', _c1='2016-01-02', _c2='20.4'),
 Row(_c0='s1', _c1='2016-01-03', _c2='55.5'),
 Row(_c0='s2', _c1='2016-01-03', _c2='52.5')]

DataFrame[_c0: string, _c1: string, _c2: string]

In [47]:
changedTypedf = df33.withColumn("_c2", df33["_c2"].cast("double"))
changedTypedf

DataFrame[_c0: string, _c1: string, _c2: double]

In [52]:
df33 = df33.orderBy("_c2", ascending = False)

In [53]:
df33.show()

+---+----------+----+
|_c0|       _c1| _c2|
+---+----------+----+
| s1|2016-01-02|60.2|
| s1|2016-01-03|55.5|
| s2|2016-01-03|52.5|
| s2|2016-01-01|30.1|
| s1|2016-01-01|20.5|
| s2|2016-01-02|20.4|
+---+----------+----+



## With SQL

In [54]:
df33.createOrReplaceTempView("ex33")


In [55]:
sqlDF = spark.sql("SELECT * FROM ex33")
sqlDF.show()

+---+----------+----+
|_c0|       _c1| _c2|
+---+----------+----+
| s1|2016-01-02|60.2|
| s1|2016-01-03|55.5|
| s2|2016-01-03|52.5|
| s2|2016-01-01|30.1|
| s1|2016-01-01|20.5|
| s2|2016-01-02|20.4|
+---+----------+----+



In [59]:
sqlDF = spark.sql("SELECT * FROM ex33 ORDER BY _c2 DESC")
sqlDF.show()

+---+----------+----+
|_c0|       _c1| _c2|
+---+----------+----+
| s1|2016-01-02|60.2|
| s1|2016-01-03|55.5|
| s2|2016-01-03|52.5|
| s2|2016-01-01|30.1|
| s1|2016-01-01|20.5|
| s2|2016-01-02|20.4|
+---+----------+----+

